<a href="https://colab.research.google.com/github/ekalachev/tensorflow/blob/master/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification

## Download a MNIST dataset

In [ ]:
!pip install -U tensorflow_datasets

## import libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_datasets as tfds

import math
import numpy as np
import matplotlib.pyplot as plt

# improve visibility for the progressbar
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

print(tf.__version__)

## Is GPU availble?

In [ ]:
tf.config.list_physical_devices('GPU')

## Load the fashion dataset

In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

## Check loaded dataset

In [ ]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples

print('Training examples: {}'.format(num_train_examples))
print('Testing examples: {}'.format(num_test_examples))

## Normalize method

In [ ]:
def normalize(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

# the 'map' method applies the 'normalize' to each element
train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

## Examining the processed data

In [ ]:
# custom labels
class_names = ['T-shirt', 'Shorts', 'Sweater', 'Dress', 'Cloak', 'Sandals', 'Shirt', 'Sneaker', 'A bag', 'Boot']

In [ ]:
# take a single image from the dataset and show it in a chart
for image, label in test_dataset.take(1):
  image = image.numpy().reshape((28, 28))

  plt.figure()
  plt.imshow(image, cmap=plt.cm.binary)
  plt.colorbar()
  plt.grid(False)
  plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
i = 0
for image, label in test_dataset.take(25):
  image = image.numpy().reshape((28, 28))
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image, cmap=plt.cm.binary)
  plt.xlabel(class_names[label])
  i += 1
plt.show()

## Build the model

Creating a neural network requires setting up layers

### Setting up layers

In [ ]:
model = tf.keras.Sequential([
  # this layer 'Flatten' converts two dimensional array to one dimensional
  # 28 and 28 means the size of image 28x28 pixels, 1 means that image has only gray shades
  # the colored image will have 3 (RGB = red, green, blue)
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),

  # this layer has 128 neurons and activation method. Activation methods decide if it should pass the calculated value to the next layer
  tf.keras.layers.Dense(128, activation=tf.nn.relu),

  # why 10 neurons? Because our dataset has only ten items of clothing. 
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

### Compile the model

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train the model

In [ ]:
BATCH_SIZE=32
train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

### Check accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset,steps=math.ceil(num_test_examples/BATCH_SIZE))
print('The accuracy: ', test_accuracy)

### Prediction

In [ ]:
for test_images, test_labels in test_dataset.take(5):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

  print(predictions.shape)
  print('Prediction for every category: ', predictions[0])
  print('Predicted category: {}, real category: {}'.format(class_names[np.argmax(predictions[0])], class_names[test_labels[0]]))